Import des packages

In [13]:
#Import des packages nécessaires dans ce notebook
import pandas as pd
import numpy as np

Lecture de la database

In [14]:
#Import puis lecture du jeu de données
df = pd.read_csv(r"C:\Users\Dylan\Desktop\Portfolio\MPG\MPG.csv")

df.head()

,Joueur,Poste,Cote,Enchère moy,Note,Note série,Note 1 an,Nb match,Nb match série,Nb match 1 an,...,Action stoppée,DMI,Club,Prochain opposant,Date,Victoire probable,%Win+12J,%Win+16J,%Win+20J,Ligue
0,Osimhen Victor,A,44,82.0,6.06,5.61,6.01,32,9,34,...,NaN,0,Napoli,NaN,NaN,NaN,NaN,NaN,NaN,SerieA
1,Martínez Lautaro,A,42,45.0,5.64,5.64,5.71,38,38,40,...,NaN,0,Inter,NaN,NaN,NaN,NaN,NaN,NaN,SerieA
2,Dia Boulaye,A,26,43.0,5.36,0.00,5.32,33,0,34,...,NaN,0,Salernitana,NaN,NaN,NaN,NaN,NaN,NaN,SerieA
3,Rafael Leão,A,34,72.0,5.83,7.00,5.92,35,3,37,...,NaN,0,Milan,NaN,NaN,NaN,NaN,NaN,NaN,SerieA
4,Lookman Ademola,A,26,NaN,5.66,5.17,5.61,31,3,32,...,NaN,0,Atalanta,NaN,NaN,NaN,NaN,NaN,NaN,SerieA


Récupération des infos de la database

In [15]:
#Récupération des infos du DF pour observer les étapes de cleaning à réaliser
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2227 entries, 0 to 2226
Data columns (total 91 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Joueur             2227 non-null   object 
 1   Poste              2227 non-null   object 
 2   Cote               2227 non-null   int64  
 3   Enchère moy        1001 non-null   float64
 4   Note               2227 non-null   float64
 5   Note série         2227 non-null   float64
 6   Note 1 an          2227 non-null   float64
 7   Nb match           2227 non-null   int64  
 8   Nb match série     2227 non-null   int64  
 9   Nb match 1 an      2227 non-null   int64  
 10  Variation          2227 non-null   float64
 11  Var série          2227 non-null   float64
 12  Var 1 an           2227 non-null   float64
 13  But                2227 non-null   int64  
 14  Buts série         2227 non-null   int64  
 15  Buts 1 an          2227 non-null   int64  
 16  %Titu              2227 

Recherche des NULL

In [16]:
#Recherche des NULL présents dans notre DF
df.isnull().sum()

Joueur                  0
Poste                   0
Cote                    0
Enchère moy          1226
Note                    0
                     ... 
Victoire probable    2227
%Win+12J             2227
%Win+16J             2227
%Win+20J             2227
Ligue                   0
Length: 91, dtype: int64

Cleaning de la database

In [17]:
#Pour commencer, nous supprimons les joueurs ayant une note égale à 0 et ceux ayant joué moins de 15 matchs
#Ces joueurs ne sont pas significatifs, nous décidons d'alléger notre DF pour faciliter le machine learning à venir
df = df[df['Note']!=0]
df = df[df['Nb match']>14]

df.reset_index(drop=True, inplace=True)

In [18]:
#On crée une colonne Valeur_Joueur qui va prendre la valeur maximum entre Cote et Enchère moy. Puis on supprime Cote et Enchère moy

df['Valeur_Joueur'] = df[['Cote', 'Enchère moy']].max(axis=1)

df = df.drop(['Cote', 'Enchère moy'], axis=1)

In [19]:
#On remplit nos NA par 0 puis on vérifie s'il nous reste des NA
df['Min/But'] = df['Min/But'].fillna(0)
df['Pass decis.'] = df['Pass decis.'].fillna(0)
df['Occas° créée'] = df['Occas° créée'].fillna(0)
df['Corner gagné'] = df['Corner gagné'].fillna(0)
df['Cleansheet'] = df['Cleansheet'].fillna(0)
df['Interceptions'] = df['Interceptions'].fillna(0)
df['Tacles'] = df['Tacles'].fillna(0)
df['%Duel'] = df['%Duel'].fillna(0)
df['Fautes'] = df['Fautes'].fillna(0)
df['But évité'] = df['But évité'].fillna(0)
df['Action stoppée'] = df['Action stoppée'].fillna(0)
df['%Passes'] = df['%Passes'].fillna(0)
df['Ballons'] = df['Ballons'].fillna(0)

df.isna().sum()

Joueur              0
Poste               0
Note                0
Note série          0
Note 1 an           0
                 ... 
%Win+12J         1494
%Win+16J         1494
%Win+20J         1494
Ligue               0
Valeur_Joueur       0
Length: 90, dtype: int64

In [20]:
#On supprime les colonnes qui ne nous serviront pas/qui sont redondantes
colonnes_serie1an= df.filter(regex='série|1 an', axis=1)
colonnes_supprimees = ['Joueur', 'Club', 'Min/But', 'Min note/but', 'Prix/but','But/Peno', 'But/Coup-franc', 'But/surface', 'Prochain opposant', 'Date', 'Victoire probable', '%Win+12J', '%Win+16J', '%Win+20J', 'DMI']
df=df.drop(columns=colonnes_supprimees)
df=df.drop(columns=colonnes_serie1an)

In [21]:
#On récupère la valeur max/min/médiane de chaque joueur
Valeur_max = df[['j1', 'j2', 'j3', 'j4', 'j5', 'j6', 'j7', 'j8', 'j9', 'j10', 'j11', 'j12', 'j13', 'j14', 'j15', 'j16', 'j17', 'j18', 'j19', 'j20', 'j21', 'j22', 'j23', 'j24', 'j25', 'j26', 'j27', 'j28', 'j29', 'j30', 'j31', 'j32', 'j33', 'j34', 'j35', 'j36', 'j37', 'j38']].max(axis=1)
Valeur_min = df[['j1', 'j2', 'j3', 'j4', 'j5', 'j6', 'j7', 'j8', 'j9', 'j10', 'j11', 'j12', 'j13', 'j14', 'j15', 'j16', 'j17', 'j18', 'j19', 'j20', 'j21', 'j22', 'j23', 'j24', 'j25', 'j26', 'j27', 'j28', 'j29', 'j30', 'j31', 'j32', 'j33', 'j34', 'j35', 'j36', 'j37', 'j38']].replace(0, float('inf')).min(axis=1)
Valeur_mediane = df[['j1', 'j2', 'j3', 'j4', 'j5', 'j6', 'j7', 'j8', 'j9', 'j10', 'j11', 'j12', 'j13', 'j14', 'j15', 'j16', 'j17', 'j18', 'j19', 'j20', 'j21', 'j22', 'j23', 'j24', 'j25', 'j26', 'j27', 'j28', 'j29', 'j30', 'j31', 'j32', 'j33', 'j34', 'j35', 'j36', 'j37', 'j38']].median(axis=1)

#On crée une colonne qui rajoute nos valeurs
df['Note_Max'] = Valeur_max
df['Note_Min_Sans_Zero'] = Valeur_min
df['Note_Mediane'] = Valeur_mediane
#On vérifie l'état de notre DF
df.head(10)

,Poste,Note,Nb match,Variation,But,%Titu,Temps,Tps moy,j38,j37,...,Tacles,%Duel,Fautes,But évité,Action stoppée,Ligue,Valeur_Joueur,Note_Max,Note_Min_Sans_Zero,Note_Mediane
0,A,6.06,32,1.38,26,0.94,2583,80.72,7.0,7.5,...,6.0,43.40,40.0,0.0,0.0,SerieA,82.0,8.5,3.0,6.00
1,A,5.64,38,1.29,21,0.71,2576,67.79,4.5,8.0,...,14.0,46.43,46.0,0.0,0.0,SerieA,45.0,8.0,3.5,5.75
2,A,5.36,33,1.39,16,0.82,2527,76.58,0.0,0.0,...,5.0,36.54,28.0,0.0,0.0,SerieA,43.0,8.0,3.0,5.00
3,A,5.83,35,1.42,15,0.80,2430,69.43,8.0,5.0,...,7.0,45.54,20.0,0.0,0.0,SerieA,72.0,8.0,3.5,5.25
4,A,5.66,31,1.17,13,0.65,1733,55.90,6.0,4.5,...,5.0,39.81,24.0,0.0,0.0,SerieA,26.0,8.0,4.0,5.00
5,A,5.20,33,1.04,13,0.76,2147,65.06,6.0,6.5,...,5.0,51.67,38.0,0.0,0.0,SerieA,19.0,8.0,3.5,5.00
6,A,4.85,31,1.40,13,0.94,2653,85.58,5.0,2.5,...,5.0,34.16,57.0,0.0,0.0,SerieA,23.0,7.5,2.5,4.00
7,A,6.16,25,1.31,12,0.84,1751,70.04,6.0,0.0,...,4.0,49.75,10.0,0.0,0.0,SerieA,64.0,8.0,4.0,5.00
8,MO,6.00,34,1.28,12,0.88,2539,74.68,6.0,4.5,...,23.0,44.79,20.0,0.0,0.0,SerieA,58.0,9.0,3.5,6.00
9,A,5.63,26,1.14,12,0.85,1872,72.00,5.5,7.0,...,14.0,45.49,33.0,0.0,0.0,SerieA,42.0,8.0,3.5,5.00


In [22]:
#On supprime les colonnes j1 à j38
colonnes_a_supprimer = ['j1', 'j2', 'j3', 'j4', 'j5', 'j6', 'j7', 'j8', 'j9', 'j10', 'j11', 'j12', 'j13', 'j14', 'j15', 'j16', 'j17', 'j18', 'j19', 'j20', 'j21', 'j22', 'j23', 'j24', 'j25', 'j26', 'j27', 'j28', 'j29', 'j30', 'j31', 'j32', 'j33', 'j34', 'j35', 'j36', 'j37', 'j38']

df.drop(columns=colonnes_a_supprimer, axis=1, inplace=True)

#On vérifie l'état de notre DF
df.head(10)

,Poste,Note,Nb match,Variation,But,%Titu,Temps,Tps moy,Cleansheet,Pass decis.,...,Tacles,%Duel,Fautes,But évité,Action stoppée,Ligue,Valeur_Joueur,Note_Max,Note_Min_Sans_Zero,Note_Mediane
0,A,6.06,32,1.38,26,0.94,2583,80.72,16.5,4.0,...,6.0,43.40,40.0,0.0,0.0,SerieA,82.0,8.5,3.0,6.00
1,A,5.64,38,1.29,21,0.71,2576,67.79,8.5,6.0,...,14.0,46.43,46.0,0.0,0.0,SerieA,45.0,8.0,3.5,5.75
2,A,5.36,33,1.39,16,0.82,2527,76.58,5.0,6.0,...,5.0,36.54,28.0,0.0,0.0,SerieA,43.0,8.0,3.0,5.00
3,A,5.83,35,1.42,15,0.80,2430,69.43,8.5,8.0,...,7.0,45.54,20.0,0.0,0.0,SerieA,72.0,8.0,3.5,5.25
4,A,5.66,31,1.17,13,0.65,1733,55.90,7.5,6.0,...,5.0,39.81,24.0,0.0,0.0,SerieA,26.0,8.0,4.0,5.00
5,A,5.20,33,1.04,13,0.76,2147,65.06,11.5,5.0,...,5.0,51.67,38.0,0.0,0.0,SerieA,19.0,8.0,3.5,5.00
6,A,4.85,31,1.40,13,0.94,2653,85.58,7.5,2.0,...,5.0,34.16,57.0,0.0,0.0,SerieA,23.0,7.5,2.5,4.00
7,A,6.16,25,1.31,12,0.84,1751,70.04,10.0,6.0,...,4.0,49.75,10.0,0.0,0.0,SerieA,64.0,8.0,4.0,5.00
8,MO,6.00,34,1.28,12,0.88,2539,74.68,14.5,10.0,...,23.0,44.79,20.0,0.0,0.0,SerieA,58.0,9.0,3.5,6.00
9,A,5.63,26,1.14,12,0.85,1872,72.00,5.0,7.0,...,14.0,45.49,33.0,0.0,0.0,SerieA,42.0,8.0,3.5,5.00


Export de la database clean

In [23]:
df.to_csv(r"C:\Users\Dylan\Desktop\Portfolio\MPG\MPG_NOT_ENCODED.csv")